# Запрос перечня друзей пользователя
Для выполнения работы импортируем необходимый пакет

In [ ]:
import vk_api

Как и при выполнении первой лабараторной работы создаем подключение, а далее находим друзей `user1` с помощью метода `friends.get`

In [ ]:
def auth_handler():
    """При двухфакторной аутентификации вызывается эта функция"""
    key = input("Enter authentication code: ")
    remember_device = True
    return key, remember_device

def stop_f(items):
    print(items)

def auth(login:str, psswd:str):
    vk_session = vk_api.VkApi(login, psswd, auth_handler=auth_handler)
    try:
        vk_session.auth()
    except vk_api.AuthError as error_msg:
        print(error_msg)
    
    return vk_session

In [ ]:
vk_session = auth("", "")
tools = vk_api.VkTools(vk_session)

In [ ]:
import time
def get_groups_users(friend_list, tools):
    friends_out = {}
    for friend in friend_list:
        try:
            friends_out[friend] = tools.get_all("friends.get", 100, {"user_id": friend})
        except Exception:
            friends_out[friend] = []
        time.sleep(1)
    return friends_out

In [ ]:
user1 = 251901521
friend_list = []
friend_list.append(user1)
friends_out = get_groups_users(friend_list, tools)
print(friends_out)

In [ ]:
print((friends_out[251901521])['items'])

In [ ]:
friends_of_friends_list = get_groups_users(friends_out[user1]['items'], tools)
print(friends_of_friends_list)

In [ ]:
list(friends_of_friends_list)
for id in list(friends_of_friends_list):
    try:
        print(len(friends_of_friends_list[id]["items"]))
    except:
        print(f"error for id:{id}")

# Формировка графа друзей пользователя
Установим модуль **networkX** (необходима версия 1.9)

In [ ]:
!pip uninstall networkx

In [ ]:
import networkx

In [ ]:
import networkx as nx

def make_graph(friends_out, friends_friends):
    graph = nx.Graph()
    graph.add_node(user1, size = friends_out[user1]['count'])
    for i in friends_out[user1]['items']:
        try:
            graph.add_node(i, size=friends_friends[i]['count'])
            intersection = set(friends_out[user1]['items']).intersection(
                set(friends_friends[i]['items']))
            graph.add_edge(user1, i, weight=len(intersection))
        except Exception:
            print("err")
    return graph